To view the sequence of items

In [ ]:
import torch

dic = torch.load('./dataset/kitchen/id2shape.pt')
seq = torch.load('./dataset/kitchen/test_sequence.pt')

print("test_sequence.pt:", seq[2])
print("id2shape.pt keys:", list(dic.keys())[:])


🔹 test_sequence.pt: [73, 149, 166, 138, 39, 91, 32, 148, 94, 71, 23, 1, 41, 124, 37, 28, 121, 113, 87, 173, 170, 52, 123, 38, 142, 126]
🔹 id2shape.pt keys: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179]


To generate a new sequence of items to be packed

In [ ]:
import torch
import os

# Load the shape dictionary
dic = torch.load('./dataset/kitchen/id2shape.pt')
num_shapes = len(dic)

# Ensure you have at least 30 shapes
assert num_shapes >= 26, f"Need at least 30 shapes, but found only {num_shapes}"

# Where to save
save_path = './dataset/kitchen/test_sequence.pt'
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Generate 500 sequences, each with 30 unique shape IDs
sequences = []
for i in range(1000):
    seq = torch.randperm(num_shapes)[:26]  # 30 unique random IDs
    sequences.append(seq.tolist())

# Save all sequences
torch.save(sequences, save_path)

print(f"✅ Created {len(sequences)} sequences, each with 26 unique shape IDs.")
print(f"Saved to: {save_path}")
print(f"Shape ID range: 0 to {num_shapes - 1}")

✅ Created 1000 sequences, each with 26 unique shape IDs.
👉 Saved to: ./dataset/kitchen/test_sequence.pt
Shape ID range: 0 to 179


Online packing to generate a dataset

In [2]:
! python main.py --evaluate --evaluation-episodes-test 30 --locmodel ./models/kitchen_online.pt --device 0 --dataset kitchen --custom blockoutexp --bufferSize 1  --num_processes 1 --samplePointsNum 1024 --selectedAction 500 --resolutionA 0.02 --resolutionH 0.01 --visual #--inference True 

pybullet build time: Dec  1 2021 18:33:04
first hierachical False
                          Options
                          hidden_size: 128
                          noisy_std: 0.5
                          atoms: 31
                          V_min: -1
                          V_max: 8
                          target_update: 1000
                          multi_step: 3
                          discount: 0.99
                          reward_clip: 0
                          learning_rate: 6.25e-05
                          batch_size: 64
                          norm_clip: 10
                          memory_capacity: 100000
                          replay_frequency: 4
                          priority_exponent: 0.5
                          priority_weight: 1.0
                          id: default
                          seed: 123
                          disable_cuda: False
                          T_max: 50000000
                          max_episode_length: 108000
   

Length of the training dataset

In [12]:
import torch
import numpy as np
pred = torch.load("/home/bakirkhon/Thesis/3D-bin-packing-master/dataset/training_dataset_irregular.pt")
print(len(pred))

657


Online predicted packing (inference=True)

In [ ]:
! python main.py --evaluate --evaluation-episodes-test 1 --locmodel ./models/kitchen_online.pt --device 0 --dataset kitchen --custom blockoutexp --bufferSize 1  --num_processes 1 --samplePointsNum 1024 --selectedAction 500 --resolutionA 0.02 --resolutionH 0.01 --visual --inference True 

pybullet build time: Dec  1 2021 18:33:04
first hierachical False
                          Options
                          hidden_size: 128
                          noisy_std: 0.5
                          atoms: 31
                          V_min: -1
                          V_max: 8
                          target_update: 1000
                          multi_step: 3
                          discount: 0.99
                          reward_clip: 0
                          learning_rate: 6.25e-05
                          batch_size: 64
                          norm_clip: 10
                          memory_capacity: 100000
                          replay_frequency: 4
                          priority_exponent: 0.5
                          priority_weight: 1.0
                          id: default
                          seed: 123
                          disable_cuda: False
                          T_max: 50000000
                          max_episode_length: 108000
   

Use this code if you always want to use a dataset of fixed length 

In [ ]:
import torch

# Load file
pred = torch.load("./dataset/training_dataset_irregular.pt")

# Slice first 500 elements
pred_small = pred[:500]

# Save to new file
torch.save(pred_small, "./dataset/training_dataset_irregular.pt")


Allows to see edge predictions in matrix form for the speciific order ID 

In [8]:
import torch
def format_sublist(subarr):
    """Convert a small [x y z] NumPy array to a string like [x, y, z]."""
    return "[" + ", ".join(f"{v:g}" for v in subarr) + "]"

def format_matrix(m):
    """Convert a 3D NumPy matrix into nested list string format with commas."""
    lines = []
    for row in m:
        row_str = ", ".join(format_sublist(sub) for sub in row)
        lines.append(f"[{row_str}],")
    return "\n".join(lines)

file_path = '/home/bakirkhon/Thesis_irregular/inference_irregular_predictions.pt'
data = torch.load(file_path)
# print(data[1]["E"].shape)
for i in range(len(data)):
    print(i)
    print(format_matrix(data[i]['E'][0]))


0
[[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]],
[[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0]],
[[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]],
[[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [